In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from IPython.display import display, clear_output

# ===============================================================
# Utility + Signal Functions
# ===============================================================
def pad_bits(bits):
    return ''.join([b for b in bits if b in '01'])

def digital_to_timebase(bits, bit_rate=1.0, samples_per_bit=200):
    n = len(bits)
    T = n / bit_rate if bit_rate else n
    total_samples = max(1, n * samples_per_bit)
    t = np.linspace(0, T, total_samples, endpoint=False)
    return t

# --- Digital Encodings ---
def unipolar_nrz(bits):
    t = digital_to_timebase(bits)
    y = np.array([1 if b == '1' else 0 for b in bits for _ in range(200)])
    return t, y

def polar_nrz(bits):
    t = digital_to_timebase(bits)
    y = np.array([1 if b == '1' else -1 for b in bits for _ in range(200)])
    return t, y

def rz(bits):
    t = digital_to_timebase(bits)
    y = np.zeros_like(t)
    half = 100
    for i, b in enumerate(bits):
        if b == '1':
            y[i*200:i*200+half] = 1
    return t, y

def manchester(bits):
    t = digital_to_timebase(bits)
    y = np.zeros_like(t)
    half = 100
    for i, b in enumerate(bits):
        if b == '1':
            y[i*200:i*200+half] = 1
            y[i*200+half:(i+1)*200] = -1
        else:
            y[i*200:i*200+half] = -1
            y[i*200+half:(i+1)*200] = 1
    return t, y

def diff_manchester(bits):
    t = digital_to_timebase(bits)
    y = np.zeros_like(t)
    half = 100
    last = -1
    for i, b in enumerate(bits):
        if b == '1':
            last = -last
        y[i*200:i*200+half] = last
        y[i*200+half:(i+1)*200] = -last
    return t, y

def bipolar_ami(bits):
    t = digital_to_timebase(bits)
    y = np.zeros_like(t)
    level = -1
    for i, b in enumerate(bits):
        if b == '1':
            level = -level
            y[i*200:(i+1)*200] = level
    return t, y

# --- Analog + Sampling ---
def generate_analog_sine(freq=2.0, amp=1.0, duration=1.0, fs=2000):
    t = np.linspace(0, duration, int(fs*duration), endpoint=False)
    x = amp * np.sin(2*np.pi*freq*t)
    return t, x

def sample_and_quantize(t, x, fsample=200, quant_levels=16):
    st = np.arange(0, t[-1], 1.0/fsample)
    svals = np.interp(st, t, x)
    xmin, xmax = -1.2*np.max(np.abs(x)), 1.2*np.max(np.abs(x))
    edges = np.linspace(xmin, xmax, quant_levels+1)
    centers = (edges[:-1] + edges[1:]) / 2
    idx = np.clip(np.digitize(svals, edges)-1, 0, quant_levels-1)
    qvals = centers[idx]
    rt = np.linspace(0, st[-1], len(t))
    recon = np.interp(rt, st, qvals)
    return st, svals, qvals, rt, recon, idx

# ===============================================================
# Plotting Logic
# ===============================================================
def plot_digital(bits, scheme):
    funcs = {
        "Unipolar NRZ": unipolar_nrz,
        "Polar NRZ": polar_nrz,
        "RZ": rz,
        "Manchester": manchester,
        "Diff Manchester": diff_manchester,
        "Bipolar AMI": bipolar_ami
    }
    if scheme not in funcs:
        print("Invalid scheme")
        return
    t, y = funcs[scheme](bits)
    plt.figure(figsize=(8,3))
    plt.plot(t, y)
    plt.title(f"{scheme} | Bits: {bits}")
    plt.grid(True)
    plt.show()

def plot_analog(freq, amp, dur):
    t, x = generate_analog_sine(freq, amp, dur)
    plt.figure(figsize=(8,3))
    plt.plot(t, x, color='green')
    plt.title(f"Sine Wave — {freq} Hz, Amp ={amp}, Dur ={dur}s")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

def plot_sampling():
    t, x = generate_analog_sine()
    st, svals, qvals, rt, recon, idx = sample_and_quantize(t, x)
    plt.figure(figsize=(8,3))
    plt.plot(t, x, 'gray', label='Analog')
    plt.stem(st, svals, linefmt='r-', markerfmt='ro', basefmt=' ')
    plt.step(rt, recon, where='post', color='orange', label='Reconstructed')
    plt.legend()
    plt.title("Sampling and Quantization")
    plt.grid(True)
    plt.show()

# ===============================================================
# Interactive UI in Colab
# ===============================================================
mode_dd = widgets.Dropdown(options=["Digital Conversion","Analog Conversion"],
                           description="Mode:")
digital_dd = widgets.Dropdown(options=["Unipolar NRZ","Polar NRZ","RZ","Manchester","Diff Manchester","Bipolar AMI"],
                              description="Digital:")
analog_dd = widgets.Dropdown(options=["Generate Sine","Sample & Quantize"],
                             description="Analog:")

bit_in = widgets.Text(value="10110", description="Bits:")
analog_in = widgets.Text(value="5,1,1", description="freq,amp,dur:")
run_btn = widgets.Button(description="Run", button_style='success')
out = widgets.Output()

def on_run(_):
    with out:
        clear_output()
        if mode_dd.value == "Digital Conversion":
            bits = pad_bits(bit_in.value)
            if bits:
                plot_digital(bits, digital_dd.value)
            else:
                print("Please enter valid bits (e.g., 10110)")
        else:
            if analog_dd.value == "Generate Sine":
                try:
                    freq, amp, dur = map(float, analog_in.value.split(','))
                    plot_analog(freq, amp, dur)
                except:
                    print("Enter as: freq,amp,duration  (e.g., 5,1,1)")
            else:
                plot_sampling()

run_btn.on_click(on_run)

ui = widgets.VBox([
    mode_dd,
    widgets.HBox([digital_dd, bit_in]),
    widgets.HBox([analog_dd, analog_in]),
    run_btn,
    out
])
display(ui)
